In [1]:
import nltk

In [2]:
nltk.download()

NLTK Downloader
---------------------------------------------------------------------------
    d) Download   l) List    u) Update   c) Config   h) Help   q) Quit
---------------------------------------------------------------------------
Downloader> d

Download which package (l=list; x=cancel)?
  Identifier> wordnet
      Unzipping corpora/wordnet.zip.

---------------------------------------------------------------------------
    d) Download   l) List    u) Update   c) Config   h) Help   q) Quit
---------------------------------------------------------------------------
Downloader> d

Download which package (l=list; x=cancel)?
  Identifier> stopwords
      Unzipping corpora/stopwords.zip.

---------------------------------------------------------------------------
    d) Download   l) List    u) Update   c) Config   h) Help   q) Quit
---------------------------------------------------------------------------
Downloader> q


True

In [3]:
!wget https://github.com/shbm/kaggle_toxic_comments/blob/master/dataset/train.csv.zip?raw=true
!wget https://github.com/shbm/kaggle_toxic_comments/blob/master/dataset/test.csv.zip?raw=true

--2018-03-05 16:45:37--  https://github.com/shbm/kaggle_toxic_comments/blob/master/dataset/train.csv.zip?raw=true
Resolving github.com (github.com)... 192.30.253.113, 192.30.253.112
Connecting to github.com (github.com)|192.30.253.113|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/shbm/kaggle_toxic_comments/raw/master/dataset/train.csv.zip [following]
--2018-03-05 16:45:37--  https://github.com/shbm/kaggle_toxic_comments/raw/master/dataset/train.csv.zip
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/shbm/kaggle_toxic_comments/master/dataset/train.csv.zip [following]
--2018-03-05 16:45:37--  https://raw.githubusercontent.com/shbm/kaggle_toxic_comments/master/dataset/train.csv.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.32.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.32.133|:4

In [4]:
!unzip test.csv.zip?raw=true
!unzip train.csv.zip?raw=true

Archive:  test.csv.zip?raw=true
  inflating: test.csv                
Archive:  train.csv.zip?raw=true
  inflating: train.csv               


In [47]:
import pandas as pd
test = pd.read_csv('test.csv')
train = pd.read_csv('train.csv')

In [6]:
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
import re


In [7]:
def lower_clean(text):
    text = str(text)
    text = text.split(' ')
    text = " ".join([w.lower() for w in text])

    return text
    
    

In [8]:
def punc(text):
    text = re.sub(r'[^A-Za-z\s]',r' ',text)
    text = re.sub(r'\n',r' ',text)    
    
    return text

In [14]:
def stop(text):
    #stop_words = set(stopwords.words('english'))
    words = text.split(' ')
    text = " ".join([w for w in words if not w in stop_words])
    return text

In [17]:
stop_words = set(stopwords.words('english'))
stop_words

{'a',
 'about',
 'above',
 'after',
 'again',
 'against',
 'ain',
 'all',
 'am',
 'an',
 'and',
 'any',
 'are',
 'aren',
 "aren't",
 'as',
 'at',
 'be',
 'because',
 'been',
 'before',
 'being',
 'below',
 'between',
 'both',
 'but',
 'by',
 'can',
 'couldn',
 "couldn't",
 'd',
 'did',
 'didn',
 "didn't",
 'do',
 'does',
 'doesn',
 "doesn't",
 'doing',
 'don',
 "don't",
 'down',
 'during',
 'each',
 'few',
 'for',
 'from',
 'further',
 'had',
 'hadn',
 "hadn't",
 'has',
 'hasn',
 "hasn't",
 'have',
 'haven',
 "haven't",
 'having',
 'he',
 'her',
 'here',
 'hers',
 'herself',
 'him',
 'himself',
 'his',
 'how',
 'i',
 'if',
 'in',
 'into',
 'is',
 'isn',
 "isn't",
 'it',
 "it's",
 'its',
 'itself',
 'just',
 'll',
 'm',
 'ma',
 'me',
 'mightn',
 "mightn't",
 'more',
 'most',
 'mustn',
 "mustn't",
 'my',
 'myself',
 'needn',
 "needn't",
 'no',
 'nor',
 'not',
 'now',
 'o',
 'of',
 'off',
 'on',
 'once',
 'only',
 'or',
 'other',
 'our',
 'ours',
 'ourselves',
 'out',
 'over',
 'own',
 'r

In [43]:
len(train)

159571

In [48]:
training = train['comment_text']
testing = test['comment_text']

training = list(training)+list(testing)
len(training)

312735

In [49]:
training = list(map(lower_clean,training))
print('lower')


lower


In [50]:
training = list(map(punc,training))
print('punc')


punc


In [51]:
for i in range(312735):
    training[i] = stop(training[i])
    if i%10000==0:
        print('stop')


stop
stop
stop
stop
stop
stop
stop
stop
stop
stop
stop
stop
stop
stop
stop
stop
stop
stop
stop
stop
stop
stop
stop
stop
stop
stop
stop
stop
stop
stop
stop


In [52]:
training[0]

'explanation edits made username hardcore metallica fan reverted  vandalisms  closure gas voted new york dolls fac  please remove template talk page since retired             '

In [55]:
COMMENT = 'comment_text'
train[COMMENT] = training[:159571]
train[COMMENT].fillna("unknown", inplace=True)
test[COMMENT] = training[159571:]
test[COMMENT].fillna("unknown", inplace=True)

In [56]:
train_features = train[COMMENT]
test_features = test[COMMENT]

In [57]:
import pandas as pd, numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

In [58]:
vec = TfidfVectorizer(ngram_range=(1,2),
               min_df=3, max_df=0.9, strip_accents='unicode', use_idf=1,
               smooth_idf=1, sublinear_tf=1 )
trn_term_doc = vec.fit_transform(train_features)
test_term_doc = vec.transform(test_features)

In [59]:
label_cols = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']

In [60]:
def pr(y_i, y):
    p = x[y==y_i].sum(0)
    return (p+1) / ((y==y_i).sum()+1)

In [61]:
x = trn_term_doc
test_x = test_term_doc

In [62]:
def get_mdl(y):
    y = y.values
    r = np.log(pr(1,y) / pr(0,y))
    m = LogisticRegression(C=4, dual=True)
    x_nb = x.multiply(r)
    return m.fit(x_nb, y), r

In [63]:
preds = np.zeros((len(test), len(label_cols)))

for i, j in enumerate(label_cols):
    print('fit', j)
    m,r = get_mdl(train[j])
    preds[:,i] = m.predict_proba(test_x.multiply(r))[:,1]

fit toxic
fit severe_toxic
fit obscene
fit threat
fit insult
fit identity_hate


In [66]:
len(preds)

153164

In [82]:
d = {'id':test['id'],'toxic':preds[:,0],'severe_toxic':preds[:,1],'obscene':preds[:,2],'threat':preds[:,3],'insult':preds[:,4],'identity_hate':preds[:,5]}

In [83]:
X = pd.DataFrame(d)

In [88]:
X.to_csv('sub.csv',index = False)

In [89]:
df = pd.read_csv('sub.csv')
df.head()

,id,identity_hate,insult,obscene,severe_toxic,threat,toxic
0,00001cee341fdb12,0.151482,0.982917,0.999874,0.036002,0.001935,0.999997
1,0000247867823ef7,0.000381,0.002702,0.000906,0.000414,0.000119,0.000548
2,00013b17ad220c46,0.000329,0.002701,0.001687,0.000263,0.000118,0.007016
3,00017563c3f7919a,0.000181,0.001367,0.001391,0.000331,0.000167,0.001396
4,00017695ad8997eb,0.000288,0.003433,0.002415,0.000361,0.000119,0.015870


In [69]:
ls

courses/   nltk_data/  sub.csv   test.csv.zip?raw=true  train.csv.zip?raw=true
examples/  README.txt  test.csv  train.csv              Untitled.ipynb


In [64]:
#submid = pd.DataFrame({'id': df["id"]})
#submission = pd.concat([submid, pd.DataFrame(preds, columns = label_cols)], axis=1)
#submission.to_csv('submission.csv', index=False)

NameError: name 'subm' is not defined